In [1]:
%matplotlib inline

# Imports

In [8]:
# Author: Till Zemann
# License: MIT License

from __future__ import annotations

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib.patches import Patch
from tqdm import tqdm

import gymnasium as gym
import gym_trading_env
from gym_trading_env.utils.history import History
from gym_trading_env.downloader import download
import datetime


# Create features

- These are the state vector inputs
- this would be an import from `TAindicators`
- only data columns starting with "`feature_`" will appear in the observation space

In [ ]:
from TAindicators import * # Import the indicators

# df is a DataFrame with columns : "open", "high", "low", "close", "Volume USD"

# this can be defined in TAindicators and imported directly to the agent
# after we get through testing things out in notebooks
def define_state_vector(df: pd.DataFrame):
    # TODO: replace these generic features with indicator functions once adapted

    # Create the feature : ( close[t] - close[t-1] )/ close[t-1]
    df["feature_close"] = df["close"].pct_change()

    # Create the feature : open[t] / close[t]
    df["feature_open"] = df["open"]/df["close"]

    # Create the feature : high[t] / close[t]
    df["feature_high"] = df["high"]/df["close"]

    # Create the feature : low[t] / close[t]
    df["feature_low"] = df["low"]/df["close"]

    # Create the feature : volume[t] / max(*volume[t-7*24:t+1])
    df["feature_volume"] = df["Volume USD"] / df["Volume USD"].rolling(7*24).max()

    df.dropna(inplace= True) # Clean again !
    # Eatch step, the environment will return 5 inputs  : "feature_close", "feature_open", "feature_high", "feature_low", "feature_volume"

# Define custom rewards and dynamic features

In [68]:
def add_reward_columns(df: pd.DataFrame):
    for col in ['lr', 'alr', 'variance', 'sr', 'powc']:
        df[col] = 0

def lr(history: History):
    """logarithmic return"""
    this_lr = 0
    # "all assets USD"
    if history['position', -1] != 0:
        this_lr = np.log(history['data_close', -1]) - np.log(history['data_close', -2])
    history.__setitem__(('data_lr', -1), this_lr) # update history with new lr
    return this_lr

def alr(history: History):
    """average logarithmic return"""

    # Using weighted incremental algorithmic approach for average
    # https://math.stackexchange.com/questions/106700/incremental-averaging
    # general formula is: mean = ((n - 1) * last_mean + this_value) / n))
    n = len(history)
    last_alr = history['data_alr', -2]
    this_lr = lr(history)
    alr = ((n - 1) * last_alr + this_lr) / n
    history.__setitem__(('data_alr', -1), alr) # update history with new alr
    return alr
    
def variance(history: History):
    """variance of logarithmic return"""
    n = len(history)
    last_alr = history['data_alr', -2]
    this_alr = alr(history)
    this_lr = history['data_lr', -1] # set when calling alr
    last_var = history['data_variance', -2]
    # TODO: finish this
    pass

def sr(history: History):
    """sharpe ratio"""
    # Using weighted incremental algorithmic approach for variance
    # https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Weighted_incremental_algorithm
    # general formula is: 
    this_alr = alr(history)
    this_lr = history['data_lr', -1] # set when calling alr
    last_var = history['data_variance', -2]
    n = len(history)
    # TODO: finish this
    pass

def powc(history):
    """profit only when closed"""
    pass

def reward_function(history):
    """Randomly weighted average of lr, alr, sr, powc"""
    pass

# Download training data

In [9]:
# Download data from 2017-01-01 to 2022-12-31
# Download BTC/USDT historical data from Binance and stores it to directory ./data/binance-BTCUSDT-1h.pkl
download(exchange_names = ["binance"],
    symbols= ["BTC/USDT"],
    timeframe= "1h",
    dir = "data",
    since = datetime.datetime(year= 2017, month= 1, day=1),
)
# Import your fresh data
df = pd.read_pickle("./data/binance-BTCUSDT-1h.pkl")
df.head()

BTC/USDT downloaded from binance and stored at data/binance-BTCUSDT-1h.pkl


,open,high,low,close,volume,date_close
date_open,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 05:00:00
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 06:00:00
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 07:00:00
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 08:00:00
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 09:00:00


In [9]:
df = pd.read_pickle('./data/binance-BTCUSDT-1h.pkl')
trainingDF = df.truncate(
    after = pd.Timestamp('2023-01-01'),
    copy = True
)

trainingDF.dropna(inplace=True)
trainingDF.sort_index(inplace=True)
trainingDF.to_csv('./data/binance_training_data.csv')

In [13]:
df = pd.read_pickle('./data/binance-BTCUSDT-1h.pkl')
testingDF = df.truncate(
    before = pd.Timestamp('2023-01-01'),
    copy = True
)

testingDF.dropna(inplace=True)
testingDF.sort_index(inplace=True)
testingDF.to_csv('./data/binance_testing_data.csv')

In [3]:
download(
    exchange_names = ["bitfinex2", "huobi"],
    symbols= ["BTC/USDT"],
    timeframe= "1h",
    dir = "data",
    since= datetime.datetime(year= 2017, month= 1, day=1)
)

BTC/USDT downloaded from huobi and stored at data/huobi-BTCUSDT-1h.pkl
BTC/USDT downloaded from bitfinex2 and stored at data/bitfinex2-BTCUSDT-1h.pkl


In [4]:
huobiDF = pd.read_pickle('./data/huobi-BTCUSDT-1h.pkl')
huobi_training = huobiDF.truncate(after='2023-01-01')
huobi_training.to_csv('./data/huobi-BTCUSDT-1h-training.csv')
huobi_test = huobiDF.truncate(before='2023-01-01')
huobi_test.to_csv('./data/huobi-BTCUSDT-1h-test.csv')

In [5]:
bitfinex2DF = pd.read_pickle('./data/bitfinex2-BTCUSDT-1h.pkl')
bitfinex2_training = bitfinex2DF.truncate(after='2023-01-01')
bitfinex2_test = bitfinex2DF.truncate(before='2023-01-01')
bitfinex2_training.to_csv('./data/bitfinex2-BTCUSDT-1h-training.csv')
bitfinex2_test.to_csv('./data/bitfinex2-BTCUSDT-1h-test.csv')

# Create Environment

In [16]:
env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = df, # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
        dynamic_feature_functions = [dynamic_feature_alr]
    )

# Define Agent

In [ ]:
class BitcoinTrainingAgent:
    def __init__(self) -> None:
        """Initialize hyperparameters"""
        pass

    def get_action(self, observation):
        """Given an observation, choose an action"""
        pass

    def update(self):
        """Update parameters"""
        pass

# Create Agent

In [ ]:
# TODO: define hyperparameters and pass to agent
bitcoinAgent = BitcoinTrainingAgent()

# Train model using agent

In [16]:
# Run an episode until it ends :
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)

Market Return : 423.10%   |   Portfolio Return : -94.73%   |   
